# Training a Sentiment Classifier

This is a notebook that trains a sentiment classifier based on data. Specifically, it uses a bag-of-words to extract features, and the structured perceptron algorithm to train the classifier.

It will take in a text `X` and return a `label` of "1" if the sentiment of the text is positive, "-1" if the sentiment of the text is negative, and "0" if the sentiment of the text is neutral. You can test the accuracy of your classifier by running the notebook all the way to end.

## Setup

Setup code, do imports.

In [15]:
import random
import tqdm

## Feature Extraction

Feature extraction code, how do we get the features we use in training? By default we just use every word.

In [16]:
def extract_features(x: str) -> dict[str, float]:
    features = {}
    x_split = x.split(' ')
    for x in x_split:
        features[x] = features.get(x, 0) + 1.0
    return features

Also, initialize the feature weights to zero.

In [17]:

feature_weights = {}

## Data Reading

Read in the data from the training and dev (or finally test) sets

In [18]:
def read_xy_data(filename: str) -> tuple[list[str], list[int]]:
    x_data = []
    y_data = []
    with open(filename, 'r') as f:
        for line in f:
            label, text = line.strip().split(' ||| ')
            x_data.append(text)
            y_data.append(int(label))
    return x_data, y_data

In [19]:
x_train, y_train = read_xy_data('./data/train.txt')
x_dev, y_dev = read_xy_data('./data/dev.txt')

In [20]:
print(x_train[0])
print(y_train[0])

The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud Van Damme or Steven Segal .
1


## Inference Code

How we run the classifier.

In [21]:
def run_classifier(features: dict[str, float]) -> int:
    score = 0
    for feat_name, feat_value in features.items():
        score = score + feat_value * feature_weights.get(feat_name, 0)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

## Training Code

Learn the weights of the classifier.

In [22]:
NUM_EPOCHS = 5
for epoch in range(1, NUM_EPOCHS+1):
    # Shuffle the order of the data
    data_ids = list(range(len(x_train)))
    random.shuffle(data_ids)
    # Run over all data points
    for data_id in tqdm.tqdm(data_ids, desc=f'Epoch {epoch}'):
        x = x_train[data_id]
        y = y_train[data_id]
        # We will skip neutral examples
        if y == 0:    
            continue
        # Make a prediction
        features = extract_features(x)
        predicted_y = run_classifier(features)
        # Update the weights if the prediction is wrong
        if predicted_y != y:
            for feature in features:
                feature_weights[feature] = feature_weights.get(feature, 0) + y * features[feature]

Epoch 5: 100%|██████████| 8544/8544 [00:00<00:00, 456412.41it/s]


## Evaluation Code

How we evaluate the classifier:

In [23]:
def calculate_accuracy(x_data: list[str], y_data: list[int]) -> float:
    total_number = 0
    correct_number = 0
    for x, y in zip(x_data, y_data):
        y_pred = run_classifier(extract_features(x))
        total_number += 1
        if y == y_pred:
            correct_number += 1
    return correct_number / float(total_number)

In [24]:
label_count = {}
for y in y_dev:
    if y not in label_count:
        label_count[y] = 0
    label_count[y] += 1
print(label_count)

{1: 444, 0: 229, -1: 428}


In [25]:
train_accuracy = calculate_accuracy(x_train, y_train)
test_accuracy = calculate_accuracy(x_dev, y_dev)
print(f'Train accuracy: {train_accuracy}')
print(f'Dev/test accuracy: {test_accuracy}')

Train accuracy: 0.7531601123595506
Dev/test accuracy: 0.5613079019073569


## Error Analysis

An important part of improving any system is figuring out where it goes wrong. The following two functions allow you to randomly observe some mistaken examples, which may help you improve the classifier. Feel free to write more sophisticated methods for error analysis as well.

In [26]:
def get_feature_contributions(features):
    output = {}
    for feat_name, feat_value in features.items():
        output[feat_name] = feat_value * feature_weights.get(feat_name, 0)
    return output

def find_errors(x_data, y_data):
    error_ids = []
    y_preds = []
    id2contributions = {}
    for i, (x, y) in enumerate(zip(x_data, y_data)):
        features = extract_features(x)
        y_preds.append(run_classifier(features))
        if y != y_preds[-1]:
            error_ids.append(i)
            id2contributions[i] = get_feature_contributions(features)
    for _ in range(5):
        my_id = random.choice(error_ids)
        x, y, y_pred = x_data[my_id], y_data[my_id], y_preds[my_id]

        print(f'{x}\ntrue label: {y}\npredicted label: {y_pred}')
        contributions = sorted(id2contributions[my_id].items(), key=lambda x: -x[1])
        for feat_name, contribution in contributions:
            print(f'Feature: {feat_name} ({contribution})')
        
        print()

In [27]:
find_errors(x_dev, y_dev)

Binoche makes it interesting trying to find out .
true label: 1
predicted label: -1
Feature: Binoche (2.0)
Feature: it (2.0)
Feature: interesting (1.0)
Feature: out (1.0)
Feature: makes (0.0)
Feature: . (-1.0)
Feature: to (-2.0)
Feature: find (-2.0)
Feature: trying (-8.0)

The lower your expectations , the more you 'll enjoy it .
true label: -1
predicted label: 1
Feature: enjoy (6.0)
Feature: you (5.0)
Feature: it (2.0)
Feature: expectations (0.0)
Feature: , (0.0)
Feature: the (0.0)
Feature: more (0.0)
Feature: lower (-1.0)
Feature: your (-1.0)
Feature: . (-1.0)
Feature: 'll (-2.0)
Feature: The (-5.0)

This movie is a snow emergency .
true label: 0
predicted label: -1
Feature: a (2.0)
Feature: emergency (1.0)
Feature: This (-1.0)
Feature: movie (-1.0)
Feature: . (-1.0)
Feature: snow (-2.0)
Feature: is (-3.0)

A great ensemble cast ca n't lift this heartfelt enterprise out of the familiar .
true label: -1
predicted label: 1
Feature: heartfelt (5.0)
Feature: great (4.0)
Feature: cast (4.

## Visualize feature weights

We can inspect the weights that were learned for various features. Below we show the largest, smallest, and randomly selected feature weights. Inspecting them may give insight into the learned classifier.

In [28]:
import random

k = 25
topk_features = sorted(feature_weights.items(), key=lambda x: -x[1])[:k]
bottomk_features = sorted(feature_weights.items(), key=lambda x: x[1])[:k]
randomk_features = random.sample(list(feature_weights.items()), k)

print("Top-k")
for feature in topk_features:
    print(feature)

print("\nBottom-k")
for feature in bottomk_features:
    print(feature)

print("\nRandom k")
for feature in randomk_features:
    print(feature)

Top-k
('solid', 14.0)
('deeply', 12.0)
('stand', 12.0)
('treat', 11.0)
('eyes', 11.0)
('rare', 11.0)
('sweet', 11.0)
('human', 11.0)
('powerful', 11.0)
('appealing', 11.0)
('remarkable', 11.0)
('ability', 11.0)
('heart', 10.0)
('fun', 10.0)
('works', 10.0)
('provides', 10.0)
('terrific', 10.0)
('summer', 10.0)
('somewhat', 10.0)
('enjoyable', 10.0)
('charming', 9.0)
('manages', 9.0)
('best', 9.0)
('beautifully', 9.0)
('definitely', 9.0)

Bottom-k
('worst', -15.0)
('mess', -14.0)
('stupid', -14.0)
('flat', -13.0)
('already', -11.0)
('none', -11.0)
('suffers', -11.0)
('failure', -11.0)
('TV', -11.0)
('Unfortunately', -11.0)
('Lawrence', -11.0)
('thinks', -11.0)
('tired', -10.0)
('figure', -10.0)
('wannabe', -10.0)
('depressing', -10.0)
('scene', -10.0)
('nowhere', -10.0)
('bland', -10.0)
('bore', -10.0)
('tiresome', -10.0)
('Bad', -10.0)
('missing', -10.0)
('were', -9.0)
('clichés', -9.0)

Random k
('hint', 2.0)
('cope', 1.0)
('Die', 0.0)
('silent-movie', 1.0)
('outrageously', -1.0)
('he